In [1]:
import great_expectations as gx
from great_expectations.data_context import FileDataContext
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
try:
    context = gx.get_context(context_root_dir = "../services/gx")
except Exception as e:
    context = FileDataContext(context_root_dir = "../services/gx")

# Add data source and data asset
data_source = context.sources.add_or_update_pandas(name="sample")
data_asset = data_source.add_csv_asset(
    name = "sample1",
    filepath_or_buffer='../data/samples/sample.csv'
)

# Create batch request
batch_request = data_asset.build_batch_request()

# Create expectation suit
context.add_or_update_expectation_suite("expectation_suite")


# Create a validator
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name="expectation_suite",
)
try:
    # Add expectations on the validator

    # Completeness & Uniqueness & Validity
    validator.expect_column_values_to_not_be_null(column='id')
    validator.expect_column_values_to_be_unique(column='id')
    validator.expect_column_values_to_be_of_type(column='id', type_='int')

    # Completeness & Consistency & Validity
    validator.expect_column_values_to_not_be_null(column='title')
    validator.expect_column_values_to_be_of_type(column='title', type_='str')
    validator.expect_column_proportion_of_unique_values_to_be_between(
        column="title",
        min_value=0.7,
        max_value=1
    )

    # Completeness & Validity & Timelessness
    validator.expect_column_values_to_not_be_null(column='lunchTime')
    validator.expect_column_values_to_match_regex_list(
        column='lunchTime',
        regex_list=[r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$', r'^(200[0-9]|201[0-9]|202[0-4])-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$']
    )

    # Completeness & Consistency & Accuracy
    validator.expect_column_values_to_not_be_null(column='sold')
    validator.expect_column_values_to_be_in_type_list(column='sold', type_list=['int'])
    validator.expect_column_min_to_be_between(column='sold', min_value=0)

    # Completeness & Consistency & Accuracy
    validator.expect_column_values_to_not_be_null(column='rating')
    validator.expect_column_values_to_be_of_type(column='rating', type_='float64')
    validator.expect_column_values_to_be_between(column='rating', min_value=0, max_value=5)

    # Completeness & Consistency & Accuracy
    validator.expect_column_values_to_not_be_null(column='discount')
    validator.expect_column_values_to_be_of_type(column='discount', type_='int')
    validator.expect_column_values_to_be_between(column='discount', min_value=0, max_value=100)

    # Completeness & Consistency & Consistency
    validator.expect_column_values_to_not_be_null(column='type')
    validator.expect_column_values_to_be_of_type(column='type', type_='str')
    validator.expect_column_distinct_values_to_equal_set(column='type', value_set=['ad', 'natural'])

    # Save expectation suite
    validator.save_expectation_suite(
        discard_failed_expectations = False
    )
    
    batch_list = data_asset.get_batch_list_from_batch_request(batch_request)

    validations = [
        {
            "batch_request": batch.batch_request,
            "expectation_suite_name": "expectation_suite"
        }
        for batch in batch_list
    ]
    checkpoint = context.add_or_update_checkpoint(
        name="validator_checkpoint",
        validations=validations
    )
    checkpoint_result = checkpoint.run()
except Exception as e:
    print(e)
# Build the data docs (website files)
context.build_data_docs()

# Open the data docs in a browser
context.open_data_docs()

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/72 [00:00<?, ?it/s]

In [3]:
from src import data
import pandas as pd
df = data.preprocess_data(pd.read_csv('../data/samples/sample.csv'))[0]
try:
    context = gx.get_context(context_root_dir="../services/gx")
except Exception as e:
    context = FileDataContext(context_root_dir="../services/gx")

# Add data source and data asset from DataFrame
data_source = context.sources.add_or_update_pandas(name="features_sample")
data_asset = data_source.add_dataframe_asset(
    name="features_sample",
    dataframe=df
)
# Create batch request
batch_request = data_asset.build_batch_request()

# Create expectation suite
context.add_or_update_expectation_suite("features_expectation_suite")

# Create a validator
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name="features_expectation_suite",
)

try:
    # Add expectations dynamically based on the column types
    for column in df.columns:
        if column.split('_')[0] in ['title','rating', 'sold', 'discount', 'shippingCost','price']:
            validator.expect_column_values_to_not_be_null(column=column)
            validator.expect_column_values_to_be_of_type(column=column, type_='float64' if df[column].dtype == 'float64'  else 'float32' if df[column].dtype=='float32' else 'int')
            #validator.expect_column_min_to_be_between(column=column, min_value=0)
        else:
            validator.expect_column_values_to_not_be_null(column=column)
            validator.expect_column_values_to_be_of_type(column=column, type_='float64' if df[column].dtype == 'float64'  else 'float32' if df[column].dtype=='float32' else 'int')
            validator.expect_column_min_to_be_between(column=column, min_value=0, max_value=1)

    # Save expectation suite
    validator.save_expectation_suite(discard_failed_expectations=False)

    batch_list = data_asset.get_batch_list_from_batch_request(batch_request)

    validations = [
        {
            "batch_request": batch.batch_request,
            "expectation_suite_name": "features_expectation_suite"
        }
        for batch in batch_list
    ]
    checkpoint = context.add_or_update_checkpoint(
        name="validator_checkpoint",
        validations=validations
    )
    checkpoint_result = checkpoint.run()
except Exception as e:
    print(e)

# Build the data docs (website files)
context.build_data_docs()

# Open the data docs in a browser
context.open_data_docs()

C:\Users\amirc\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,

cpu
C:\Users\amirc\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: resume_download is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use force_download=True.
  warnings.warn(



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 2/2 [00:54<00:00, 27.23s/it]


     rating      sold  discount  shippingCost  category_name_access-control  \
0 -0.778876 -0.150239  0.400314     -0.262995                           0.0   
1 -0.778876 -0.150239 -0.697591      0.566471                           0.0   
2 -0.778876 -0.147651  0.400314     -0.064987                           0.0   
3 -0.778876 -0.150239  2.888896     -0.166154                           0.0   
4 -0.778876 -0.150239 -0.697591      0.566471                           0.0   
5 -0.778876 -0.150239 -0.697591      0.566471                           0.0   
6  1.337734  0.582410  0.839475     -0.262995                           0.0   
7 -0.778876 -0.150239  1.790992     -0.173641                           0.0   
8  1.337734 -0.132117 -0.112042     -0.207622                           0.0   
9 -0.778876 -0.150239  0.253926     -0.173495                           0.0   

   category_name_action-toy-figures  category_name_art-supplies  \
0                               0.0                         0.0

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

	2048 expectation(s) included in expectation_suite.
	2048 expectation(s) included in expectation_suite.


Calculating Metrics:   0%|          | 0/4871 [00:00<?, ?it/s]